### Download dependencies

In [ ]:
# Note: Must install kaggle first:
#       Download .tar.gz file from Pypi
#       Unzip compressed file
#       Run setup using `python -m setup.py install`

# Install additional packages
!pip install opendatasets --upgrade --quiet

### Downloading the data

Download the data package from Kaggle to a temporary directory; the temp dir is created via `tempfile` and the dataset download is handled via `opendatasets`. Note Kaggle asks for a username and key -- type "a" into both.

In [ ]:
import os
import tempfile
import opendatasets as od

# Create new temporary directory
data_folder = tempfile.mkdtemp()

# Download dataset into temporary directory
# A more complete dataset can be found at 'https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023'
dataset = 'https://www.kaggle.com/datasets/usdot/flight-delays'
od.download(dataset, data_folder, force=True)

### Data element insights

The following categories are given in xx:yy - hour:minute format (e.g. 1536 means 3:36pm, 345 means 3:45am, 16 means 00:16am): 
- `SCHEDULED_DEPARTURE`
- `DEPARTURE_TIME`
- `SCHEDULED_ARRIVAL`
- `ARRIVAL_TIME`
- `WHEELS_OFF`
- `WHEELS_ON`

Other parameters are given in minutes (negative values indicate the actual time occurred before the scheduled time by the same magnitude):
- `ARRIVAL_DELAY`
- `DEPARTURE_DELAY`
- `TAXI_IN`
- `TAXI_OUT`
- `SCHEDULED_TIME`
- `ELAPSED_TIME`
- `AIR_TIME`

The `DISTANCE` parameter is given in miles.

Specific definitions of some parameters are as follows:
- `WHEELS_OFF`: The time point that the aircraft's wheels leave the ground.
- `WHEELS_ON`: The time point that the aircraft's wheels touch on the ground.
- `TAXI_OUT`: The time duration elapsed between departure from the origin airport gate and wheels off.
- `TAXI_IN`: The time duration elapsed between wheels-on and gate arrival at the destination airport.
- `AIR_TIME`: The time duration between wheels_off and wheels_on time.

And, similarly, relationships between different data columns can be defined:
- `ARRIVAL_TIME` = `WHEELS_ON` + `TAXI_IN`
- `ARRIVAL_DELAY` = `ARRIVAL_TIME` - `SCHEDULED_ARRIVAL`
- `DEPARTURE_TIME` = `WHEELS_OFF` - `TAXI_OUT`
- `DEPARTURE_DELAY` = `DEPARTURE_TIME` - `SCHEDULED_DEPARTURE`
- `ELAPSED_TIME` = `AIR_TIME` + `TAXI_IN` + `TAXI_OUT`
- `AIR_TIME` = `WHEELS_ON` - `WHEELS_OFF`

### Airlines dataset

Let's look at the Airlines dataframe. In structure, it's pretty straight-forward, comprising two columns: the International Air Transport Association (IATA) code for each airline, and the name of the respective airline.

In [ ]:
import pandas as pd

# Read each CSV dataset
airlines_path = os.path.join(data_folder,'flight-delays','airlines.csv')

# Read each CSV into Pandas dataframe
airlines = pd.read_csv(airlines_path,low_memory=False)

print(airlines.head())

### Airports dataset

We need information on which airports the airlines operate from and where each airport is located. Each airport has an associated IATA code.

In [ ]:
# Read each CSV dataset
airports_path = os.path.join(data_folder,'flight-delays','airports.csv')

# Read each CSV into Pandas dataframe
airports = pd.read_csv(airports_path,low_memory=False)

print(airports.head())

### Flights dataset

The remaining file contains data for each flight described by the date of each flight, flight specifics (tail number, airline, and flight number), and the flight's origin and destination information. The data also include flight diversion, delay, and cancellation status. If the flights were delayed, the dataset provides the delay type and delay length. Note that January 1, 2015 (`MONTH`=1, `DAY`=1, `YEAR`=2015) was a Thursday; thus the first day of the week (`DAY_OF_WEEK`=1) corresponds to Monday.

In [ ]:
# Read each CSV dataset
flights_path = os.path.join(data_folder,'flight-delays','flights.csv')

# Read each CSV into Pandas dataframe
flights = pd.read_csv(flights_path,low_memory=False)
pd.set_option('display.max_columns', None)
print(flights.head())

### Split data: Cancelled VS Non-cancelled ("successful")
We'll start by splitting all recorded flights into cancelled flights and non-cancelled (or 'successful') flights, only focusing on the latter throughout the remainder of the tutorial.

In [ ]:
# Split dataset into cancelled flights and uncancelled flights
flown_flights = flights[flights['CANCELLED']==0]
cancl_flights = flights[flights['CANCELLED']==1]

### Cleaning / homogenizing flight data
We'll then simply the resulting dataframe further:

- Successful flights were not cancelled, so we'll remove the `CANCELLED` and `CANCELLATION_REASON` columns.
- `TAIL_NUMBER` and `FLIGHT_NUMBER` will not impact the delay type, so we'll get rid of those.
- `ARRIVAL_DELAY` accounts for `ARRIVAL_TIME` and `SCHEDULED_ARRIVAL`, so keep the former and remove the latter.
- `ARRIVAL_TIME` accounts for `WHEELS_ON` and `TAXI_IN`, so keep the former and remove the latter.
- `DEPARTURE_DELAY` accounts for `DEPARTURE_TIME` and `SCHEDULED_DEPARTURE`; keep the former, remove the latter.
- `DEPARTURE_TIME` accounts for `WHEELS_OFF`, and `TAXI_OUT`; keep the former, remove the latter.
- `ELAPSED_TIME` accounts for `AIR_TIME` and `DISTANCE` since longer distances require longer air time. Keep the former, remove the latter.
- `ARRIVAL_DELAY` is likely the sum of the other delay types (`DEPARTURE`,`AIR_SYSTEM`,`SECURITY`,`LATE_AIRCRAFT`, and `WEATHER`) but we cannot be sure. Remove `ARRIVAL_DELAY` and define a new column to represent the delay sum.
- Rearrange all columns to group `_DELAY` types .

Coincidentally, dropping these eight columns also eliminates every column of a format other than minutes or miles.

In [ ]:
# Remove cancellation-related columns
clean_flown = flown_flights.drop(columns=['CANCELLED','CANCELLATION_REASON'])

# Remove irrelevant flight information (tail, flight number)
clean_flights = clean_flown.drop(columns=['TAIL_NUMBER','FLIGHT_NUMBER'])

# Remove redundant arrival data
clean_arrival = clean_flights.drop(columns=['ARRIVAL_TIME','SCHEDULED_ARRIVAL','WHEELS_ON','TAXI_IN'])

# Remove redundant departure data
clean_departure = clean_arrival.drop(columns=['DEPARTURE_TIME','SCHEDULED_DEPARTURE','WHEELS_OFF','TAXI_OUT'])

# Remove redundant delay summation data
clean_arrival = clean_departure.drop(columns=['ARRIVAL_DELAY','AIR_TIME','DISTANCE'])

# Calculate the total delay for each flight
clean_arrival['TOTAL_DELAY'] = clean_arrival['DEPARTURE_DELAY'] + clean_arrival['AIR_SYSTEM_DELAY'] + clean_arrival['SECURITY_DELAY'] + clean_arrival['AIRLINE_DELAY'] + clean_arrival['LATE_AIRCRAFT_DELAY'] + clean_arrival['WEATHER_DELAY']

# Rearrange columns to group similar concepts
clean_arrival = clean_arrival[['YEAR','MONTH','DAY','DAY_OF_WEEK','AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT',
                               'SCHEDULED_TIME','ELAPSED_TIME','DIVERTED', 'DEPARTURE_DELAY','AIR_SYSTEM_DELAY',
                               'SECURITY_DELAY','AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY','WEATHER_DELAY','TOTAL_DELAY']]

# Show current dataframe
clean_arrival.head()

### Handling date/time

Parameters like "YEAR", "MONTH", and "DAY" can be combined into "Day_of_Year", which has the added benefit of tracking seasonality and possibly interesting metrics later on.

In [ ]:
from datetime import datetime

# Combine YEAR, MONTH, and DAY into a single column
clean_arrival['DAY_OF_YEAR'] = [datetime(y,m,d).timetuple().tm_yday for y,m,d in zip(clean_arrival['YEAR'],
                                                                                     clean_arrival['MONTH'],
                                                                                     clean_arrival['DAY'])]
# Consolidate calendar information
clean_doy = clean_arrival.drop(columns=['YEAR','MONTH','DAY'])

# Print output metrics
print('\n# rows:', clean_doy.shape[0], '\n# categories:',clean_doy.shape[1])
print('\n',clean_doy.head())

### Airline consistency

We still need to handle airline specification. Currently, airlines are specified by uninterpretable 2-letter acronyms (ex. "NK"). We'll bring in other datasets to convert the acronyms to human-readable labels (ex. 'Spirit Airlines').

In [ ]:
# Generate dictionaries from dataframes
airline_dict = dict(zip(airlines.IATA_CODE, airlines.AIRLINE))

# Update airline name
airline_name = []
[airline_name.append(airline_dict[u]) for u in clean_doy['AIRLINE']]
clean_doy['AIRLINE'] = airline_name

print(clean_doy.head())

### Origin / Destination inconsistencies

Airport identifiers are also problematic. For example, `ORIGIN_AIRPORT` and `DESTINATION_AIRPORT` both contain 3-letter IATA airport codes (ex. "LAX") and 5-digit FAA numerical codes (ex. 12789). Both formats are valid, but their formats are inconsistent.

The numerical FAA airport identifiers are not included in the original Kaggle dataset, but they can be connected using two additional datafiles provided by the Bureau of Transportation Statistics (`L_AIRPORT_ID.csv` and `L_AIRPORT.csv`). The first links the numerical ID value to an aiport name, and the second links the airport name to its alphabetic airport identifier.

In [ ]:
# Define paths for files connecting airport IDs and IATA codes
id_path = os.path.join(os.getcwd(),'flight-delays','L_AIRPORT_ID.csv')
code_path = os.path.join(os.getcwd(),'flight-delays','L_AIRPORT.csv')

# Read CSV files as dataframes
airport_id = pd.read_csv(id_path, low_memory=False)
airport_code = pd.read_csv(code_path, low_memory=False)

# Merge dataframes on Airport name
merged_codes = pd.merge(airport_id, airport_code, on="Description")

# Rename dataframe columns
merged_codes = merged_codes.rename(columns={"Code_x": "FAA_ID", "Code_y": "IATA"})

# Show new results
merged_codes.head()

With this "Rosetta stone" in hand, we correct the `flights` dataset to contain only IATA codes in both columns. 

In [ ]:
print('\nOld airport origin list:\n', clean_doy.ORIGIN_AIRPORT.unique())

# Create dictionary linking airport IDs with IATA codes
orig_airport_dict = dict(zip(merged_codes.FAA_ID, merged_codes.IATA))

# Match airport ID numbers with IATA codes
orig_airport = []
for a in clean_doy.ORIGIN_AIRPORT:
    if str(a).isdecimal():
        orig_airport.append(orig_airport_dict[int(a)])
    else:
        orig_airport.append(a)

# Replace dataframe column
flown_origin = clean_doy.copy()
flown_origin.ORIGIN_AIRPORT = orig_airport

# Test if all elements in ORIGIN_AIRPORT are alphabetic identifiers
print('\nNew airport origin list:\n',flown_origin.ORIGIN_AIRPORT.unique())
print("\nAll elements only contain letters:",all([x.isalpha() for x in flown_origin.ORIGIN_AIRPORT]),'\n')

Repeat the same steps to convert `DESTINATION_AIRPORT` values

In [ ]:
# Match airport ID numbers with IATA codes
dest_airport = []
for d in flown_origin.DESTINATION_AIRPORT:
    if str(d).isdecimal():
        dest_airport.append(orig_airport_dict[int(d)])
    else:
        dest_airport.append(d)

# Replace dataframe column
flown_dest = flown_origin.copy()
flown_dest.DESTINATION_AIRPORT = dest_airport

# Test if all elements in DESTINATION_AIRPORT are alphabetic identifiers
print("\nAll elements only contain letters:",all([x.isalpha() for x in flown_dest.DESTINATION_AIRPORT]),'\n')

Manually remove outliers in `DESTINATION_AIRPORT`. Specifically, the airport "BSM" was recently updated to "AUS", and this switch does not exist in our current lookup tables so we make the change manually. Then we check there are no more list differences between our known airports and those listed in our agnostically processed dataframe.

In [ ]:
# Show difference between `flights` IATA codes and `airports` IATA codes
print('\nAirport list differences:')
print('   Before substitution:', len(list(set(flown_dest.ORIGIN_AIRPORT) - set(airports.IATA_CODE))))

# Handle each IATA code change
flown_dest.ORIGIN_AIRPORT = flown_dest.ORIGIN_AIRPORT.replace('BSM','AUS')
flown_dest.DESTINATION_AIRPORT = flown_dest.DESTINATION_AIRPORT.replace('BSM','AUS')

# Show output
print('    After substitution:', len(list(set(flown_dest.ORIGIN_AIRPORT) - set(airports.IATA_CODE))))

### Add information to dataframes based on IATA codes

Based on our knowledge of each airport's location, we can define latitude and longitude for each origin and destination airport.

In [ ]:
import numpy as np

# Copy the existing dataframe before adding information
flown_lla = flown_dest.copy()

# Create dictionaries comprising origin airport, latitude, and longitude
lats_dict = dict(zip(airports.IATA_CODE, airports.LATITUDE))
lons_dict = dict(zip(airports.IATA_CODE, airports.LONGITUDE))

# ---------- Add Latitude and Longitude for Origin ----------
# Append matching lat,lon for each Origin_Airport
olats,olons = [],[]
[olats.append(lats_dict[u]) for u in flown_lla['ORIGIN_AIRPORT']]
[olons.append(lons_dict[u]) for u in flown_lla['ORIGIN_AIRPORT']]

# Add origin location to dataframe
flown_lla['ORIGIN_LAT'] = olats
flown_lla['ORIGIN_LON'] = olons

# ---------- Add Latitude and Longitude for Destination ----------
# Append matching lat,lon for each Destination_Airport
dlats,dlons = [],[]
[dlats.append(lats_dict[u]) for u in flown_lla['DESTINATION_AIRPORT']]
[dlons.append(lons_dict[u]) for u in flown_lla['DESTINATION_AIRPORT']]

# Add origin location to dataframe
flown_lla['DESTINATION_LAT'] = dlats
flown_lla['DESTINATION_LON'] = dlons

# Show all columns when printing dataframes
pd.set_option('display.max_columns', None)

# Since the LLA values make the airport names redundant, ORIGIN_AIRPORT and DESTINATION_AIRPORT can be removed
flown_lla = flown_lla.drop(columns=['ORIGIN_AIRPORT','DESTINATION_AIRPORT'])
print(flown_lla.head())

The last troublesome issue is the `NaN` values in our `_DELAY` columns; since our ML model will be used to predict the `_DELAY` type, this is a big deal. To avoid any incorrect assumptions, we'll remove all rows where any type of `_DELAY` value is `NaN` (Not a Number). Based on the remaining `_DELAY` values, we iterate through each flight and figure out which delay is the leading cause of the flight's delay, but we'll doing so knowing that when no delays are recorded, the `MAX_DELAY` values are set accordingly.

#### A discussion on removing NaN delay values
This topic turns into its own discussion. While further investigation reveals that if all NaN values are removed from each `_DELAY` column, there are no more flights in the remaining dataset that experience a culmination of 0.0 errors for each delay type -- that is, there are no "perfect" on-time flights left. It is tempting to assume that flights containing `NaN` values for each `_DELAY` indicates that there were no delays, and these values can be set to 0. But this cannot be the case, since the minimum and maximum values for `DEPARTURE_DELAY` are -82.0 and 1988.0, respectively within the existing dataframe, and the `DEPARTURE_DELAY`s had to be due to some reason (ex. weather, airlines, late aircraft, etc.), and there is no corresponding match. Additionally, when no delay occurs in any of the `_DELAY` columns, the values are often listed as `0.0` instead of `NaN`, so it is unsafe to assume that `NaN` values indicate `0.0` for all `_DELAY` columns.

After these changes, we are left with 1,063,439 flights characterized by 19 categories.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Drop rows with NaN values in _DELAY columns
flown_noNA = flown_lla.dropna(subset = ['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])

# Find max delay type in each row
sub_df = flown_noNA[['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']]
max_delay = sub_df.idxmax(axis=1)
flown_noNA.loc[:,'MAX_DELAY'] = max_delay

# Ensure no delay values are accurately recorded
num_noDelay = len(flown_noNA[
                          (flown_noNA['AIR_SYSTEM_DELAY'] == 0.0) &
                          (flown_noNA['SECURITY_DELAY'] == 0.0) &
                          (flown_noNA['AIRLINE_DELAY'] == 0.0) &
                          (flown_noNA['LATE_AIRCRAFT_DELAY'] == 0.0) &
                          (flown_noNA['WEATHER_DELAY'] == 0.0)
                          ])
print('Number of flights with 0 delays: ',num_noDelay)

# Show sum for each max_delay column value
print('\nNumber of delay counts for each type:\n',flown_noNA['MAX_DELAY'].value_counts())

# Show head of remaining dataframe
print('\nRemaining data shape: ',flown_noNA.shape)
print('\n',flown_noNA.head())

# Initial data exploration

   Let's begin by casting a very wide analysis net by plotting histograms of all features within the dataset to get an idea of their distributions. Note that some parameters (`INDEX`, `FLIGHT_NUMBER`, `DIVERTED`) do not tell us much, while other plots tell us a great deal (ex. `DAY_OF_WEEK`, `SCHEDULED_TIME`, `TOTAL_DELAY`).

In [ ]:
flown_noNA.hist(bins=100, figsize=(20,15))
print(flown_noNA.head())

## Delay time VS. Airlines

   Let's take this one step further and dig into the `TOTAL_DELAY` as a function of airlines. We'll first begin by looking at the total delay time for each airline. To do so, we will plot the total delay time for each airline on the same plot using the Seaborn package. By sorting the values in descending order, airlines with the highest delays will listed first, and airlines with the shortest delays will be listed last. To handle extreme cases (e.g. outliers), we will plot the logarithm of the total delay time, and sort on the log(Delay Time) values.

   The Seaborn plot below demonstrates in 2015, American Airlines flights experienced the highest delay time of the 14 airlines included in the database.

In [ ]:
print(flown_noNA.head())

In [ ]:
import seaborn as sns

sns.set_theme(style="dark", rc={'figure.figsize':(11,5)})

# Pre-emptively sort dataframe
sorted_delay = flown_noNA.sort_values(by='TOTAL_DELAY',ascending=False).reset_index()
descending_delays = sorted_delay['AIRLINE'].unique()

# Generate plot of delays sorted by decreasing delay
log_delay = np.log(sorted_delay['TOTAL_DELAY'])
delay_plot = sns.violinplot(sorted_delay, x="AIRLINE", y=log_delay, order=descending_delays)
delay_plot.set_xticklabels(delay_plot.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.xlabel("Airline")
plt.ylabel("log(Total delay)")

# Show head of dataframe
print(sorted_delay.head())

## Delay time VS Day of the Week

   In a similar manner, Seaborn can be used to plot the log(Total Delay) values for each day of the week; keep in mind that in this dataset, `DAY_OF_WEEK`=1 corresponds to a Monday.

   The plot below demonstrates that traveling on the fifth day of the week -- Friday -- is associated with the longest flight delays overall. Following Friday, Saturday and Sunday are associated with the next longest flight delays, with weekdays bringing up the lowest delays. Since Friday and the weekends are generally associated with higher volume traffic in aiports, we can reasonably associate the increase in the number of flights with the increase in delays.

In [ ]:
# Pre-emptively sort dataframe
descending_DayOfWeek = sorted_delay['DAY_OF_WEEK'].unique()

# Generate plot of delays sorted by decreasing delay
delay_plot = sns.violinplot(sorted_delay, x="DAY_OF_WEEK", y=log_delay, order=descending_DayOfWeek)
delay_plot.set_xticklabels(delay_plot.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.xlabel("Day of Week")
plt.ylabel("log(Total delay)")

   But is this really the case? To make an association between the number of flights and the total length of flight delays, we'd need to examine the number of flights recorded on each day of the week. The number of flights can be plotted against the day of the week using the dataframe's `sum` function.

   Interestingly, the largest number of flights within the dataset occurred on Thursday (DoW=4), then Monday (DoW=1), *then* Friday (DoW=5). Based on this comparison, we can say that the length of the flight delays is not directly associated with the *number* of flights on a given day, but rather some other factor. Of course, we must keep in mind that these conclusions are reached on a dataset of decreased volume based on the cleaning we performed at the beginning of the analysis.

In [ ]:
num_flights = [(sorted_delay['DAY_OF_WEEK']==x).sum() for x in np.arange(1,8)]
plt.figure(figsize=(6,3))
plt.bar(np.arange(1,8), num_flights)
plt.xlabel('Day of Week')
plt.ylabel('Number of flights')
plt.show()

## Delay cause

   Alternatively, we can query the database directly to find the root cause of the delays. We can use the cleaned dataset to plot the delay type according to the database for the leading offender in flight delays.

   The results illustrate that "Airline" is the leading cause, followed by late aircraft then weather. While not terribly helpful, the plot does show that late flights and weather are amongth the top three causes of flight delays. Nonetheless, questions remain: What is constituted as an "Airline delay"? What causes "late aircraft"? How are weather and late aircraft associated? These are questions that remain outside the scope of this course given our interest in developing an AI/ML approach to data analysis.

In [ ]:
# Generate plot of delay type for most delayed airline
largest_delay_airline = sorted_delay['AIRLINE'][0]
largest_delay_by_airline = sorted_delay[sorted_delay['AIRLINE']==largest_delay_airline]

# Retain only flight delay data
trimmed_df = largest_delay_by_airline[['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY','TOTAL_DELAY']]
delay_melt = trimmed_df.melt('TOTAL_DELAY', var_name='TYPE', value_name='LENGTH')

# Sort data by delay length
sorted_by_delay_len = delay_melt.sort_values(by='LENGTH',ascending=False).reset_index()
sorted_by_delay_len['TYPE'] = [x.replace('_DELAY','').replace('_',' ').capitalize() for x in sorted_by_delay_len['TYPE']]
descending_Type = sorted_by_delay_len['TYPE'].unique()
sorted_by_delay_len['LENGTH'] = np.log(sorted_by_delay_len['LENGTH'])

# Display violin plot of delay type sorted by decreasing delay length
delay_plot = sns.violinplot(sorted_by_delay_len, x="TYPE", y="LENGTH",order=descending_Type)
plt.xlabel("Delay type")
plt.ylabel("log(Delay length)")
plt.title(largest_delay_airline + " delay causes")

Other detail-oriented analyses could also be done:
- Delay causes across each airline
- Delay length different geographic regions
- Delay cause across different geographic regions
- etc.

# Imputation

One way to resolve the missing data issue might be to use SciKit Learn's `impute` function to predict `NaN` datapoints based on a K-Nearest Neighbor algorithm. Although this method is shown below, it takes a very long time to execute given the size of our dataframe. Therefore, we'll eliminate rows with NaN values to see what size of dataframe we end up with. Nonetheless, code to impute the missing values is retained here for completeness.

In [ ]:
'''
from sklearn.impute import KNNImputer

# create a temporary copy of the dataset
flights_temp = flown_no_sum.copy()

# We'll have trouble with string values; remove for now
flights_temp = flights_temp.drop(columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT'])

# retrieve columns with numerical data; will exclude the ocean_proximity column since the datatype is object; other columns are float64
columns_list = [col for col in flights_temp.columns if flights_temp[col].dtype != 'str']

# extract columns that contain at least one missing value
new_column_list = [col for col in flights_temp.loc[:, flights_temp.isnull().any()]]

# update temp dataframe with numeric columns that have empty values
flights_temp = flights_temp[new_column_list]

# initialize KNNImputer to impute missing data using machine learning
knn = KNNImputer(n_neighbors = 3)

# fit function trains the model 
knn.fit(flights_temp)

# Transform the data using the model; applies the transformation model (ie knn) to data
array_Values = knn.transform(flights_temp)

# convert the array values to a dataframe with the appropriate column names
flights_temp = pd.DataFrame(array_Values, columns = new_column_list)
'''

### Extract largest delay type

Ultimately, our Machine Learning model will attempt to predict the largest delay type for a given flight provided all other flight parameters. Therefore, we need to drop `NaN` values within the _DELAY columns, and isolate the largest delay type for each flight. This will allow us to count the frequency of each delay type to ensure the ML model can utilize evenly distributed data.

## Cancelled flight analysis
We'll now start cleaning and analyzing the cancelled flights. 

We can begin cleaning the dataframe by saving useful data columns by replacing confusing data with informal data. For instance, `CANCELLATION_REASONS` A, B, C, and D stand for "Airline", "Weather", "National Air System", and "Security", respectively. In this context, `NaN` likely indicates the flight was not cancelled, so these values can be replaced by "None" (as a string). We do this knowing we can use ML methods later on to categorize flights based on the cancellation reason.

In [ ]:
## Remove _DELAY columns from cancelled flights, and make cancellation reasons readable
#clean_cancl = cancl_flights.drop(columns=['ARRIVAL_DELAY','DIVERTED','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
#clean_cancl = clean_cancl.drop(columns=['DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN'])
#clean_cancl = clean_cancl.drop(columns=['ARRIVAL_TIME'])

## Make cancellation reason intelligible
#clean_cancl.loc[:,'CANCELLATION_REASON'] = clean_cancl['CANCELLATION_REASON'].map({'A': 'Airline', 'B': 'Weather','C':'National Air System', 'D':'Security'}) 
#clean_cancl['CANCELLATION_REASON'].unique()

#cancl_nulls = clean_cancl.isnull()
#cancl_nulls_sum = cancl_nulls.sum().to_dict()
#print('Size of cancelled flight dataframe: ',clean_cancl.shape)
#print('\nNaN values in each dataset:\n',clean_cancl.isnull().sum())

In [ ]:
## Show total number of data points
#print('\nTotal number of data points:',clean_cancl.shape[0])
#
## Query for null parameters in dataset
#print('\nNaN values in each dataset:\n',clean_cancl.isnull().sum())
#nulls_df = clean_cancl.isnull()
#null_sum = nulls_df.sum().to_dict()

The column containing the most NaN values appears to be "CANCELLATION REASON". We'll identify this column programmatically and remove it from the dataframe. Then, we'll remove all other rows that contain NaN values. This will allow us to work with a dataframe void of NaN values with just over one million datapoints.

Save information for Cancellation Reason

In [ ]:
#clean_flights['CANCELLATION_REASON'].unique()

In [ ]:
## Find max null set, remove column
#max_null_count = max(nulls_df.sum())
#res = [key for key in null_sum if null_sum[key] == max_null_count]
#ncr_df = flights.drop(columns=res)
#print("\nLargest set with NaN:",res[0])

## Drop all other rows containing NaN
#clean_flights = ncr_df.dropna().reset_index()
#print("\nTotal remaining NaN values:", sum(clean_flights.isnull().sum()))

# AI/ML Analysis

With some rudimentary analysis in the background, we'll now begin our dataset analysis using AI/ML. Specifically, we'll try to develop a model to categorize the *type* of delay experienced by a flight given the timing of the flight (day of week, year of day), airline, and other factors.

### Parameter relationships
Plotting the correlation of each numerical value (`numeric_only=True`) in the dataframe shows us how strongly each variable is related to other variables within the same dataframe. We immediately notice the `DIVERTED` dataset has null correlation; further investigation shows this is because there is one unique value in `DIVERTED` (0). We'll remove it for now.

### Create categorization
To do this, we'll add one more column combining all five `_DELAY` columns into one column that describes the maximum delay experienced for any given flight.

In [ ]:
# Plot a graphical correlation matrix for each pair of columns in the dataframe
corr1 = sorted_delay.corr(numeric_only=True)

plt.figure(figsize = (8,6))
sns.heatmap(corr1, annot=False)
sns.set(font_scale=0.7)
plt.show()

# Remove `DIVERTED`
after_clean = sorted_delay.drop(columns=['index','DIVERTED'])

# Plot a graphical correlation matrix for each pair of columns in the dataframe
corr2 = after_clean.corr(numeric_only=True)

plt.figure(figsize = (8,6))
sns.heatmap(corr2, annot=False)
sns.set(font_scale=0.7)
plt.show()

In [ ]:
# Generate correlation between numerical values and airline names
airlines = after_clean['AIRLINE'].unique()
indices = np.arange(len(airlines))
airline_key = zip(airlines,indices)

# Show corresponding values
print(list(airline_key))

# Show result beforehand
print('\nUnique airlines before substitution:\n',after_clean['AIRLINE'].unique())

for k in np.arange(len(airlines)):
    after_clean.loc[after_clean['AIRLINE'] == airlines[k], 'AIRLINE'] = k

# Show result afterward
print('\nUnique airlines after substitution:\n',after_clean['AIRLINE'].unique())

In [ ]:
print(after_clean.head())

In [ ]:
# Generate correlation between numerical values and airline names
delays = after_clean['MAX_DELAY'].unique()
indices = np.arange(len(delays))
delay_key = zip(delays,indices)

# Create unique labels for ML characterization
uniqlbl = [x.split('_')[0] for x in after_clean['MAX_DELAY'].unique()]

for k in np.arange(len(delays)):
    after_clean.loc[after_clean['MAX_DELAY'] == delays[k], 'MAX_DELAY'] = k

# Remove any final NaN values from dataframe
clean_noNA = after_clean.dropna()

print("\nFinal dataset size:\n",clean_noNA.shape)
print('\nNaN values in final array:\n',clean_noNA.isnull().values.any(),'\n')

In [ ]:
uniqlbl

### Split the tests

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import utils

# Splitting the data into independent and dependent variables
X = clean_noNA.drop(columns=['MAX_DELAY']).values
y = clean_noNA['MAX_DELAY'].values
feature_list = list(clean_noNA.drop(columns=['MAX_DELAY']).columns)

# Creating the Training and Test set from data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

lab = preprocessing.LabelEncoder()
y_train = lab.fit_transform(y_train)
y_test = lab.fit_transform(y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier

# --------- PLOT TOP ROW OF CLASSIFIERS ---------
classifiers = {
    "Dec. Tree": DecisionTreeClassifier(),
    "Rnd Forest": RandomForestClassifier(n_estimators = 25, criterion = 'gini'),
    "Naive Bayes": GaussianNB(),
    "Log. Reg.": LogisticRegression(),
}

f, axes = plt.subplots(1, 4, figsize=(12,6), sharey='row')

log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for i, (key, classifier) in enumerate(classifiers.items()):
    y_pred = classifier.fit(X_train, y_train).predict(X_test)
    cf_matrix = confusion_matrix(y_test, y_pred, normalize='true')
    
    disp = ConfusionMatrixDisplay(cf_matrix,
                                  display_labels=uniqlbl,
                                 )
    disp.plot(ax=axes[i], xticks_rotation=45)
    disp.ax_.set_title(key)
    disp.im_.colorbar.remove()
    disp.ax_.set_xlabel('')
    if i!=0:
        disp.ax_.set_ylabel('')

plt.subplots_adjust(wspace=0.40, hspace=0.1)
plt.tight_layout()
plt.show()

In [ ]:
import warnings
import pandas as pd

# Suppress warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

classifiers = {
    "Dec. Tree": DecisionTreeClassifier(),
    "Rnd Forest": RandomForestClassifier(n_estimators = 25, criterion = 'gini'),
    "Naive Bayes": GaussianNB(),
    "Log. Reg.": LogisticRegression(),
}

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for i, (key, classifier) in enumerate(classifiers.items()):
    y_pred = classifier.fit(X_train, y_train).predict(X_test)
    cf_matrix = confusion_matrix(y_test, y_pred)

    #name = classifier.__class__.__name__    
    name = key
    train_predictions = classifier.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    train_predictions = classifier.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = pd.concat([log, log_entry], ignore_index=True)


# Image bar charts for Accuracy and Log Loss
import seaborn as sns
plt.figure(figsize=(15,2))
plt.subplot(1,2,1)
plt.style.use('dark_background')
sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")
plt.xlabel('Accuracy %')
plt.title('Classifier Accuracy')

plt.subplot(1,2,2)
sns.set_color_codes("muted")
sns.barplot(x='Log Loss', y='Classifier', data=log, color="g")
plt.xlabel('Log Loss')
plt.title('Classifier Log Loss')
plt.show()

In [ ]:
# Image bar charts for Accuracy and Log Loss
import seaborn as sns
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
plt.style.use('dark_background')
sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")
plt.xlabel('Accuracy %')
plt.title('Classifier Accuracy')

plt.subplot(1,2,2)
sns.set_color_codes("muted")
sns.barplot(x='Log Loss', y='Classifier', data=log, color="g")
plt.xlabel('Log Loss')
plt.title('Classifier Log Loss')
plt.show()